In [1]:
"""
SageMath code to check CCZ-equivalence between G_a and G_1 (Li-Kaleyski)
for small field sizes where computation is feasible.
"""

from itertools import product
import time

def build_G_function(F, q, a):
    """
    Build the function G_a(x,y,z) = (x^{q+1} + a*x^q*z + y*z^q, 
                                      x^q*z + y^{q+1}, 
                                      x*y^q + a*y^q*z + z^{q+1})
    """
    def G(x, y, z):
        comp1 = x^(q+1) + a * x^q * z + y * z^q
        comp2 = x^q * z + y^(q+1)
        comp3 = x * y^q + a * y^q * z + z^(q+1)
        return (comp1, comp2, comp3)
    return G

def function_to_matrix(F, field_size):
    """
    Convert a function F: F_q^3 -> F_q^3 to its graph matrix representation.
    The graph is {(x,F(x)) : x in F_q^3} subset of F_q^6.
    """
    points = []
    for x in field_size:
        for y in field_size:
            for z in field_size:
                fx, fy, fz = F(x, y, z)
                points.append((x, y, z, fx, fy, fz))
    return points

def compute_extended_walsh_transform(F, G, field_size):
    """
    Compute a simplified version of the extended Walsh transform
    to detect potential CCZ-equivalence.
    """
    # This is a simplified indicator - full CCZ test is more complex
    walsh_values = {}
    
    for a in field_size:
        for b in field_size:
            for c in field_size:
                sum_val = 0
                for x in field_size:
                    for y in field_size:
                        for z in field_size:
                            fx1, fy1, fz1 = F(x, y, z)
                            fx2, fy2, fz2 = G(x, y, z)
                            # Simplified Walsh-like transform
                            if (fx1 == fx2 and fy1 == fy2 and fz1 == fz2):
                                sum_val += 1
                
                if sum_val not in walsh_values:
                    walsh_values[sum_val] = 0
                walsh_values[sum_val] += 1
    
    return walsh_values

def check_EA_equivalence(F, G, field):
    """
    Check if F and G are EA-equivalent (Extended Affine equivalent).
    This is a necessary condition for CCZ-equivalence.
    """
    field_size = field.cardinality()
    
    # Check if they have the same differential uniformity
    delta_F = compute_differential_uniformity(F, field)
    delta_G = compute_differential_uniformity(G, field)
    
    if delta_F != delta_G:
        return False, "Different differential uniformities"
    
    # Check if they have the same nonlinearity (simplified check)
    nl_F = compute_nonlinearity_indicator(F, field)
    nl_G = compute_nonlinearity_indicator(G, field)
    
    if nl_F != nl_G:
        return False, "Different nonlinearity indicators"
    
    return True, "Possibly EA-equivalent (further checks needed)"

def compute_differential_uniformity(F, field):
    """
    Compute the differential uniformity of F.
    """
    max_solutions = 0
    
    # Sample check for computational efficiency
    sample_size = min(field.cardinality(), 10)
    
    for alpha in field:
        for beta in field:
            for gamma in field:
                if alpha == 0 and beta == 0 and gamma == 0:
                    continue
                
                # Count maximum solutions over a sample of outputs
                for _ in range(sample_size):
                    u, v, w = field.random_element(), field.random_element(), field.random_element()
                    
                    solution_count = 0
                    for x in field:
                        for y in field:
                            for z in field:
                                fx1, fy1, fz1 = F(x, y, z)
                                fx2, fy2, fz2 = F(x + alpha, y + beta, z + gamma)
                                
                                if (fx2 - fx1 == u and fy2 - fy1 == v and fz2 - fz1 == w):
                                    solution_count += 1
                    
                    max_solutions = max(max_solutions, solution_count)
                    
                    if max_solutions > 2:  # Early exit for non-APN
                        return max_solutions
    
    return max_solutions

def compute_nonlinearity_indicator(F, field):
    """
    Compute a simplified nonlinearity indicator.
    """
    # This is a very simplified version - actual nonlinearity computation is complex
    indicator = 0
    
    sample_size = min(field.cardinality()^2, 20)
    
    for _ in range(sample_size):
        a = (field.random_element(), field.random_element(), field.random_element())
        b = (field.random_element(), field.random_element(), field.random_element())
        
        count = 0
        for x in field:
            for y in field:
                for z in field:
                    fx, fy, fz = F(x, y, z)
                    # Simplified linear approximation check
                    if (a[0]*x + a[1]*y + a[2]*z == b[0]*fx + b[1]*fy + b[2]*fz):
                        count += 1
        
        indicator += count
    
    return indicator

def check_ccz_candidates(m, i, test_values=None):
    """
    Main function to check CCZ-equivalence candidates.
    
    Parameters:
    m: field extension degree
    i: exponent for q = 2^i
    test_values: list of a values to test (if None, tests all)
    """
    if gcd(i, m) != 1:
        print(f"Warning: gcd({i}, {m}) != 1")
        return
    
    F = GF(2^m, 'w')
    q = 2^i
    
    print(f"\n{'='*70}")
    print(f"CCZ-EQUIVALENCE CHECK FOR m={m}, i={i}, q={q}")
    print(f"Field: F_{{2^{m}}} with {F.cardinality()} elements")
    print(f"{'='*70}")
    
    # Build Li-Kaleyski function (a=1)
    G_LK = build_G_function(F, q, F(1))
    
    # Determine which values to test
    if test_values is None:
        # For small fields, test all; for larger fields, test a sample
        if F.cardinality() <= 32:
            test_values = [a for a in F if a != 0]
        else:
            # Test a representative sample
            test_values = [F.primitive_element()^k for k in range(0, F.cardinality()-1, (F.cardinality()-1)//10)]
            test_values = [a for a in test_values if a != 0][:10]
    
    results = {
        'equivalent': [],
        'possibly_equivalent': [],
        'not_equivalent': []
    }
    
    for a in test_values:
        if a == 0:
            continue
        
        print(f"\nTesting a = {a}...")
        
        # Build G_a
        G_a = build_G_function(F, q, a)
        
        # Quick check: if a^{q+1} = 1, they should be affinely equivalent
        if a^(q+1) == 1:
            print(f"  Note: a^{{q+1}} = 1, so G_a is affinely equivalent to G_1")
            results['equivalent'].append(a)
            continue
        
        # Check EA-equivalence indicators
        ea_equiv, msg = check_EA_equivalence(G_a, G_LK, F)
        
        if not ea_equiv:
            print(f"  Not CCZ-equivalent: {msg}")
            results['not_equivalent'].append(a)
        else:
            print(f"  Possibly CCZ-equivalent: {msg}")
            results['possibly_equivalent'].append(a)
            
            # For small fields, do more extensive checking
            if F.cardinality() <= 8:
                walsh_a = compute_extended_walsh_transform(G_a, G_LK, F)
                walsh_1 = compute_extended_walsh_transform(G_LK, G_LK, F)
                
                if walsh_a == walsh_1:
                    print(f"    Walsh signatures match!")
                else:
                    print(f"    Walsh signatures differ")
    
    # Summary
    print(f"\n{'='*70}")
    print("SUMMARY")
    print(f"{'='*70}")
    print(f"Total values tested: {len(test_values)}")
    print(f"Affinely equivalent (a^{{q+1}}=1): {len(results['equivalent'])}")
    if results['equivalent']:
        print(f"  Values: {results['equivalent']}")
    print(f"Possibly CCZ-equivalent: {len(results['possibly_equivalent'])}")
    if results['possibly_equivalent']:
        print(f"  Values: {results['possibly_equivalent']}")
    print(f"Not CCZ-equivalent: {len(results['not_equivalent'])}")
    
    return results

def test_small_cases():
    """
    Test CCZ-equivalence for small cases where computation is feasible.
    """
    test_cases = [
        (3, 1),  # m=3, i=1, q=2
        (3, 2),  # m=3, i=2, q=4
    ]
    
    all_results = {}
    
    for m, i in test_cases:
        print(f"\n{'#'*70}")
        print(f"TESTING CASE: m={m}, i={i}")
        print(f"{'#'*70}")
        
        results = check_ccz_candidates(m, i)
        all_results[(m, i)] = results
    
    return all_results

# Additional function to check specific values
def check_specific_values(m, i, a_values):
    """
    Check CCZ-equivalence for specific a values.
    
    Example usage:
    check_specific_values(3, 1, [F(w), F(w^2), F(w+1)])
    where F = GF(8, 'w') and w is primitive element
    """
    F = GF(2^m, 'w')
    
    # Convert a_values to field elements if needed
    field_values = []
    for a in a_values:
        if a in F:
            field_values.append(F(a))
        else:
            print(f"Warning: {a} is not in the field")
    
    return check_ccz_candidates(m, i, field_values)

# Run the tests
if __name__ == "__main__":
    print("="*70)
    print("CCZ-EQUIVALENCE CHECKER FOR G_a FUNCTIONS")
    print("="*70)
    print("\nThis program checks whether G_a is CCZ-equivalent to G_1 (Li-Kaleyski)")
    print("Note: Full CCZ-equivalence testing is computationally intensive.")
    print("This implementation provides necessary condition checks and indicators.")
    
    # Run tests for small cases
    results = test_small_cases()
    
    # Example of checking specific values for m=5
    # Uncomment to run (this will take longer)
    """
    print("\n" + "#"*70)
    print("TESTING SPECIFIC VALUES FOR m=5")
    print("#"*70)
    
    F = GF(32, 'w')
    w = F.primitive_element()
    
    # Test the 11 permutation values from Theorem 4.5
    perm_values = [
        w^3, w^3 + w, w^3 + w^2 + w,
        w^4 + w^3 + w^2 + w + 1, w^4 + w + 1,
        w^3 + w^2 + w + 1, w^4 + w^3 + w^2 + w,
        w^3 + w + 1, w^3 + 1, w^4 + w, F(1)
    ]
    
    results_m5 = check_specific_values(5, 1, perm_values[:5])  # Test first 5 for speed
    """

CCZ-EQUIVALENCE CHECKER FOR G_a FUNCTIONS

This program checks whether G_a is CCZ-equivalent to G_1 (Li-Kaleyski)
Note: Full CCZ-equivalence testing is computationally intensive.
This implementation provides necessary condition checks and indicators.

######################################################################
TESTING CASE: m=3, i=1
######################################################################

CCZ-EQUIVALENCE CHECK FOR m=3, i=1, q=2
Field: F_{2^3} with 8 elements

Testing a = w...
  Not CCZ-equivalent: Different nonlinearity indicators

Testing a = w^2...
  Not CCZ-equivalent: Different nonlinearity indicators

Testing a = w + 1...
  Not CCZ-equivalent: Different nonlinearity indicators

Testing a = w^2 + w...
  Not CCZ-equivalent: Different nonlinearity indicators

Testing a = w^2 + w + 1...
  Possibly CCZ-equivalent: Possibly EA-equivalent (further checks needed)
    Walsh signatures differ

Testing a = w^2 + 1...
  Not CCZ-equivalent: Different nonlinearity indi